In [42]:
import glob
import numpy as np
from functools import reduce
import os
import subprocess
from Bio import SeqIO 
from Bio import AlignIO

In [78]:
def SummariseGene(gene):
    if os.path.isfile('SEQ_summary.csv'):
        stat_file = open('SEQ_summary.csv', 'a')
    else:
        stat_file = open('SEQ_summary.csv', 'w')
        stat_file.write(','.join(['Gene','N_raw','N_cur','N_par','N_species_tree','Curator_raw','Curator_cur'])+'\n')  
    
    raw_files = glob.glob('SEQUENCES/' + gene + '/*RAW*.fasta')
    curated_files = glob.glob('SEQUENCES/' + gene + '/*CURATED*.fasta')
    n_par_files = len(glob.glob('SEQUENCES/' + gene + '/*PARTIAL*.fasta'))
    
    # check raw files 
    n_raw_files = len(raw_files)
    if n_raw_files > 0:
        if n_raw_files == 1:
            raw_curator = raw_files[0].split('-')[1]
            species_raw = [rec.description for rec in SeqIO.parse(open(raw_files[0]), "fasta")]
        else:
            raw_curator = 'More than one file'
            species_raw = []

    # check curated files
    n_curated_files = len(curated_files)
    if n_curated_files > 0:
        if n_curated_files == 1:
            cur_curator = curated_files[0].split('-')[1]
            species_curated = [rec.description for rec in SeqIO.parse(open(curated_files[0]), "fasta")]
        else:
            cur_curator = 'More than one file'
            species_curated = []
    else:
        cur_curator = 'No file'
        species_curated = []
    
    # create the files for the tree pipeline
    raw_and_curated = list(reduce(np.intersect1d, [species_raw,species_curated,]))
    n_for_analysis = len(raw_and_curated)
    
    dmel_seq = [rec for rec in SeqIO.parse(open('SEQUENCES/DMEL/DMEL_ref_genes.fasta'), "fasta") if rec.description == gene]
    dmel_seq = dmel_seq[0]
    dmel_seq.id = ''
    dmel_seq.description = 'Drosophila_melanogaster'
    
    clean_raw_seqs = [rec for rec in SeqIO.parse(open(raw_files[0]), "fasta") if rec.description in raw_and_curated]
    clean_cur_seqs = [rec for rec in SeqIO.parse(open(curated_files[0]), "fasta") if rec.description in raw_and_curated]
    clean_raw_seqs.append(dmel_seq)
    clean_cur_seqs.append(dmel_seq)
    
    if os.path.isdir('TREES'):
        SeqIO.write(clean_raw_seqs, 'TREES/' + gene + '_raw_seq.fa', 'fasta')
        SeqIO.write(clean_cur_seqs, 'TREES/' + gene + '_cur_seq.fa', 'fasta')
    else:
        os.mkdir('TREES')
        SeqIO.write(clean_raw_seqs, 'TREES/' + gene + '_raw_seq.fa', 'fasta')
        SeqIO.write(clean_cur_seqs, 'TREES/' + gene + '_cur_seq.fa', 'fasta')
    
    stat_file.write(','.join([gene,str(n_raw_files),str(n_curated_files),str(n_par_files),str(n_for_analysis),raw_curator,cur_curator])+'\n')
    stat_file.close()
    return(raw_and_curated)

def CreateTC(gene):
    wd = '~/Desktop/CMS/TC/TREES/'
    with open('log_file.log','a') as log:
        log.write(gene + ' - starting...\n')
        try:
            # mafft
            args = ['ginsi','TREES/' + gene + '_raw_seq.fa','>','TREES/' + gene + '_aln_raw_seq.fa']
            log.write(' '.join(args) + '\n')
            subprocess.call(' '.join(args), shell=True)
            
            args = ['ginsi','TREES/' + gene + '_cur_seq.fa','>','TREES/' + gene + '_aln_cur_seq.fa']
            log.write(' '.join(args) + '\n')
            subprocess.call(' '.join(args), shell=True)

            # trimal
            args = ['trimal', '-automated1', '-in', 'TREES/' + gene + '_aln_raw_seq.fa','-out','TREES/' + gene + '_trim_raw_seq.fa']
            log.write(' '.join(args) + '\n')
            subprocess.call(' '.join(args), shell=True)
            
            args = ['trimal', '-automated1', '-in', 'TREES/' + gene + '_aln_cur_seq.fa','-out','TREES/' + gene + '_trim_cur_seq.fa']
            log.write(' '.join(args) + '\n')
            subprocess.call(' '.join(args), shell=True)

            # raxml tree
            args = ['raxmlHPC', '-p', '12345', '-m', 'PROTGAMMAWAG', '-#', '100', '-s', 'TREES/' + gene + '_trim_raw_seq.fa', '-f', 'a', '-x', '12345',
                    '-w', wd, '-n', gene + '_tree_raw', '-o', 'Drosophila_melanogaster']
            log.write(' '.join(args) + '\n')
            subprocess.call(' '.join(args), shell=True)
            
            args = ['raxmlHPC', '-p', '12345', '-m', 'PROTGAMMAWAG', '-#', '100', '-s', 'TREES/' + gene + '_trim_cur_seq.fa', '-f', 'a', '-x', '12345',
                    '-w', wd, '-n', gene + '_tree_cur', '-o', 'Drosophila_melanogaster']
            log.write(' '.join(args) + '\n')
            subprocess.call(' '.join(args), shell=True)
            
            # raxml tc
            args = ['raxmlHPC', '-b', '12345', '-m', 'PROTGAMMAWAG', '-#', '100', '-f', 'i', 
                    '-n', 'TC', '-w', wd, '-z', 'TREES/RAxML_bootstrap.' + gene + '_tree_raw', '-t', 'TREES/RAxML_bestTree.' + gene + '_tree_raw', '-L', 'MR']
            log.write(' '.join(args) + '\n')
            subprocess.call(' '.join(args), shell=True)
            args = ['raxmlHPC', '-b', '12345', '-m', 'PROTGAMMAWAG', '-#', '100', '-f', 'i', 
                    '-n', 'TC', '-w', wd, '-z', 'TREES/RAxML_bootstrap.' + gene + '_tree_cur', '-t', 'TREES/RAxML_bestTree.' + gene + '_tree_cur', '-L', 'MR']
            log.write(' '.join(args) + '\n')
            subprocess.call(' '.join(args), shell=True)
            log.write(gene + ' DONE\n')
            return()
        except:
            log.write(gene + 'ERROR\n')
            return()

def GeneStats(gene):
    if os.path.isfile('SEQ_stats.csv'):
        stat_file = open('SEQ_stats.csv', 'a')
    else:
        stat_file = open('SEQ_stats.csv', 'w')
        stat_file.write(','.join(['Gene','Type','len_sd','trimmed_length','tree_certainty'])+'\n')  
    
    # raw
    seq_len_sd_raw = statistics.stdev([len(rec) for rec in SeqIO.parse('TREES/' + gene + '_raw_seq.fa', 'fasta')])
    trimmed_length_raw = AlignIO.read('TREES/' + gene + '_aln_raw_seq.fa', 'fasta').get_alignment_length()   
    with open('RAxML_info.' + gene + '_tree_raw') as tc_file:
        for line in tc_file.readlines():
            if line.startswith("Relative tree certainty for this tree:"):
                tc_raw = line.split(" ")[-1]
    stat_file.write(','.join([gene,'raw',str(seq_len_sd_raw),str(trimmed_length_raw),str(tc_raw)])+'\n')
    
    seq_len_sd_raw = statistics.stdev([len(rec) for rec in SeqIO.parse('TREES/' + gene + '_cur_seq.fa', 'fasta')])
    trimmed_length_raw = AlignIO.read('TREES/' + gene + '_aln_cur_seq.fa', 'fasta').get_alignment_length()   
    with open('RAxML_info.TC' + gene + '_tree_raw') as tc_file:
        for line in tc_file.readlines():
            if line.startswith("Relative tree certainty for this tree:"):
                tc_raw = line.split(" ")[-1]
    stat_file.write(','.join([gene,'cur',str(seq_len_sd_cur),str(trimmed_length_cur),str(tc_cur)])+'\n')
    stat_file.close()
    return()

def ProcessGene(gene):
    SummariseGene(gene)
    CreateTC(gene)
    GeneStats(gene)

In [ ]:
seq_files = glob.glob('SEQUENCES/*/*.fasta')
genes = list(set([i.split('/')[1] for i in seq_files]))
#for gene in genes:
#    ProcessGene(gene)
ProcessGene('DEF1')